<a href="https://colab.research.google.com/github/Mrprey/UFAL/blob/master/Anderson_Pibic20_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Bibliotecas



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import vectorize, float64, njit, jit, int32
from sklearn.manifold import MDS,TSNE
import time
import sys
from sklearn.utils.extmath import randomized_svd
from scipy.spatial.distance import cdist

# LAMP

Local Affine Multidimensional Projection


---


[Repositorio do autor](https://github.com/lgnonato/LAMP)


In [ ]:
# Author: Luis Gustavo Nonato  -- <gnonato@icmc.usp.br>
# License: BSD 3 - Clause License 2018

# This is an implementation of the technique described in:
# http://www.lcad.icmc.usp.br/~nonato/pubs/lamp.pdf
epsilon = 1e-7

class Lamp():
    def __init__(self, Xdata = None, control_points = None, weights = None, label=False, scale=True, dim = 2):
        '''
          Xdata: N_by_K matrix where N is the number of instances and K is the dimension
          control_points: M_by_(dim + 1) matrix with the coordinates of the control points
                        in the image space. The last column contains the ids of the control
                        points in the Xdata matrix
          weights: N_by_M matrix where weights[i,j] is the weight between Xdata[i] and
                   control_point[j]. If weights=None the inverse of the Euclidean distance is used as weight
          label: when True assumes the last column of Xdata as labels
          scale: apply a transformation in the control points before accomplish the mapping.
                 Produce better projections when the original and control points coordiantes has very different scales.
          dim: dimension of the image space
        '''
        self.data = None
        self.mapped = None
        self.control_points = None
        self.control_points_data = None
        self.weights = weights
        self.dim = dim
        self.labels = None
        self.label = label
        self.scale = scale
        self.data_center = None
        self.control_points_center = None
        self.U = None
        self.S = None
        self.V = None

        if Xdata is not None:
            try:
                if type(Xdata) is not np.ndarray:
                    raise TypeError()

                if label is False:
                    self.center(Xdata,o_c_s='o')
                else:
                    self.center(Xdata[:,:-1],o_c_s='o')
                    self.labels = Xdata[:,-1].astype(int)

            except (AttributeError, TypeError):
                print('----- LAMP Error -----')
                print('Xdata must be a Numpy Array')
                sys.exit()

        if control_points is not None:
            try:
                if type(control_points) is not np.ndarray:
                    raise TypeError()

            except (AttributeError, TypeError):
                print('----- LAMP Error -----')
                print('control_points must be a Numpy Array')
                sys.exit()

            self.center(control_points,o_c_s='c')


#########
    def fit(self,Xdata=None):
        ''' decide if the mapping should be trigger with or without control points'''
        if Xdata is not None:
            try:
                if type(Xdata) is not np.ndarray:
                    raise TypeError()

                if self.data is not None:
                    if self.label is True:
                        self.center(Xdata[:,:-1],o_c_s='s')
                        self.labels = Xdata[:,-1].astype(int)
                    else:
                        self.center(Xdata,o_c_s='s')
                else:
                    if self.label is True:
                        self.center(Xdata[:,:-1],o_c_s='o')
                        self.labels = Xdata[:,-1].astype(int)
                    else:
                        self.center(Xdata,o_c_s='o')

            except (AttributeError, TypeError):
                print('----- LAMP Error -----')
                print('Type Error: Xdata must be a Numpy Array')
                sys.exit()
        else:
            try:
                if self.data is None:
                    raise ValueError()

            except ValueError:
                print('----- LAMP Error -----')
                print('No data to map')
                sys.exit()

        if self.control_points is not None:
            if (self.scale is True) and (self.S is None):
                self.scale_control_points()

            self.map()

            if self.scale is True:
                self.unscale()

            mapped = self.mapped + self.control_points_center
            if self.label is True:
                mapped = np.hstack((mapped,self.labels.reshape(self.labels.shape[0],1)))
        else:
            self.control_points_free_mapping()  # To be implemented soon
            mapped = self.mapped


        return(mapped)

#########
    def map(self):
        ''' mapping using control points '''
        n,k = self.data.shape
        m = self.control_points.shape[0]
        self.mapped = np.zeros((n,self.dim))
        ctp_mapped = self.control_points[:,:-1]
        ctp_data = self.control_points_data

        if self.weights is None:
            self.weights = cdist(self.data,ctp_data,p=2.)
            self.weights = 1.0/(self.weights+epsilon)

        for i in range(n):
            alpha = np.sum(self.weights[i])
            x_tilde = np.dot(ctp_data.T,self.weights[i].T)/alpha
            y_tilde = np.dot(ctp_mapped.T,self.weights[i].T)/alpha
            x_hat = ctp_data - x_tilde
            y_hat = ctp_mapped - y_tilde
            D = np.diag(np.sqrt(self.weights[i]))
            A = np.dot(D,x_hat)
            B = np.dot(D,y_hat)
            U,s,V = randomized_svd(np.dot(A.T,B), n_components=2, random_state=None)
            M = np.dot(U,V)
            self.mapped[i] = np.dot((self.data[i] - x_tilde),M)+y_tilde

#########
    def center(self,X,o_c_s='o'):
        '''o_c_s distriminate between original data (o), control points (c), and streaming data (s)'''
        if o_c_s == 'o':
            Xmean = np.average(X, axis=0)
            self.data_center = Xmean
            self.data = np.subtract(X,Xmean)

        if o_c_s == 'c':
            ctp_ids = X[:,-1].astype(int)
            self.control_points_data = self.data[ctp_ids]
            Xmean = np.average(X[:,0:2], axis=0)
            self.control_points_center = Xmean
            centered = np.subtract(X[:,0:2],Xmean)
            self.control_points = np.hstack((centered,X[:,-1].reshape(-1,1)))

        if o_c_s == 's':
            self.data = np.subtract(X,self.data_center)

#########
    def scale_control_points(self):
        ctp_ids = self.control_points[:,-1].astype(int)
        ctp_mapped = self.control_points[:,:-1]
        ctp_data = self.data[ctp_ids]
        Uo,So,Vo = randomized_svd(ctp_data.T, n_components=2, random_state=None)
        Uc,Sc,Vc = randomized_svd(ctp_mapped.T, n_components=2, random_state=None)
        DSo = np.diag(So)
        self.control_points[:,:-1] = np.dot(np.dot(Uc,DSo),Vc).T

        self.U = Uc
        self.S = np.diag(Sc)
        self.So = np.diag(So)
        self.V = Vc

#########
    def unscale(self):
        proj = np.dot(self.U.T,self.mapped.T)
        Sinv = np.zeros((2,2))
        Sinv[0,0] = 1.0/self.So[0,0]
        Sinv[1,1] = 1.0/self.So[1,1]
        proj_unscaled = np.dot(Sinv,proj)
        self.mapped = np.dot(self.U,np.dot(self.S,proj_unscaled)).T

#########
    def control_points_free_mapping(self):
        ''' Mapping without control points '''
        def stress(p,pt,d):
            cost = 0
            for i in range(3):
                cost += ((p[0]-pt[i,0])**2+(p[1]-pt[i,1])**2-d[i])**2
            return(cost)

        max_dist = 1e8
        n = self.data.shape[0]
        knn = 5
        self.mapped = np.zeros((n,self.dim))

        D = cdist(self.data,self.data,p=2.)

        # finding the 3 first points that give rise to the inicial triangle

        ### starting with the farthest 2 points
        idx_flat = np.argmax(D)
        idx_i = idx_flat//n
        idx_j = idx_flat - n*idx_i

        ### starting with 2 random
        #idx_i = int(np.random.randint(low=0,high=n,size=1))
        #idx_j = int(np.random.randint(low=0,high=n,size=1))
        #while (idx_j == idx_i):
        #    idx_j = int(np.random.randint(low=0,high=n,size=1))

        d01 = D[idx_i,idx_j]
        d01 += epsilon
        D[idx_i,idx_j] = 0
        D[idx_j,idx_i] = 0
        processed_points = [idx_i,idx_j]

        D_processed = D[processed_points]
        idx_flat = np.argmax(D_processed)
        idx_i = idx_flat//n
        idx_j = idx_flat - n*idx_i
        d02 = D[processed_points[0],idx_j]**2 + epsilon
        d12 = D[processed_points[1],idx_j]**2 + epsilon
        D[processed_points,idx_j] = 0
        D[idx_j,processed_points] = 0
        processed_points.append(idx_j)

        # computing the initial triangle
        x = (-d12+d02+d01**2)/(2.0*d01)
        y = np.sqrt(d02-x**2)
        ltemp = [(0,0),(d01,0),(x,y)]
        proj_tri = np.asarray(ltemp)
        self.mapped[processed_points] = proj_tri[:]


        # computing k more initial points
        for i in range(3,knn):
            D_processed = D[processed_points]
            idx_flat = np.argmax(D_processed)
            idx_i = idx_flat//n
            idx_j = idx_flat - n*idx_i
            d = np.asarray([D[processed_points[0],idx_j]**2+epsilon, D[processed_points[1],idx_j]**2+epsilon, D[processed_points[2],idx_j]**2+epsilon])
            D[processed_points,idx_j] = 0
            D[idx_j,processed_points] = 0
            processed_points.append(idx_j)

            res = minimize(stress, np.asarray([0,0]),args=(proj_tri,d))
            self.mapped[idx_j] = res.x

        D = D + max_dist*np.identity(n)
        for idx_j in processed_points:
            D[processed_points,idx_j] = max_dist
            D[idx_j,processed_points] = max_dist

        # mapping the data set
        weights = np.zeros((knn,))
        for i in range(knn,n):
            D_processed = D[processed_points]
            idx_flat = np.argmin(D_processed)
            idx_i = idx_flat//n
            idx_j = idx_flat - n*idx_i

            neighbors_ids = np.argpartition(D[idx_j,processed_points],knn-1)[:knn]
            ctp_ids = [processed_points[j] for j in neighbors_ids]
            weights[:] = D[idx_j,ctp_ids]
            weights = 1.0/(weights+epsilon)
            ctp_data = self.data[ctp_ids]
            ctp_mapped = self.mapped[ctp_ids]

            alpha = np.sum(weights)
            x_tilde = np.dot(ctp_data.T,weights.T)/alpha
            y_tilde = np.dot(ctp_mapped.T,weights.T)/alpha
            x_hat = ctp_data - x_tilde
            y_hat = ctp_mapped - y_tilde
            S = np.diag(np.sqrt(weights))
            A = np.dot(S,x_hat)
            B = np.dot(S,y_hat)
            U,s,V = randomized_svd(np.dot(A.T,B), n_components=2, random_state=None)
            M = np.dot(U,V)
            self.mapped[idx_j] = np.dot((self.data[idx_j] - x_tilde),M)+y_tilde

            D[processed_points,idx_j] = max_dist
            D[idx_j,processed_points] = max_dist
            processed_points.append(idx_j)

def LAMPx(df_sample):
    sample_size = 7
    samples = np.random.randint(0, high=df_sample.shape[0], size=(sample_size,))

    # Projetar os pontos de Controle com o MDS

    ctp_mds = MDS(n_components=2)
    ctp_samples = ctp_mds.fit_transform(df_sample[samples]-np.average(df_sample[samples]))

    # Inclui os pontos de controle na ultima coluna
    ctp_samples = np.hstack((ctp_samples, samples.reshape(sample_size,1)))

    # Executando a LAMP

    lamp_proj = Lamp(Xdata = df_sample, control_points=ctp_samples, label=True)
    data_sample_proj = lamp_proj.fit()
    return data_sample_proj

#obtendo os dados

Data set original

In [ ]:
color_histogram = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/ColorHistogram.data',skiprows=4, sep=";", index_col=0)
mammals = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/mammals-50000.bin-normcols.data',skiprows=4, sep=";", index_col=0)
fibers = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/fibers.data',skiprows=4, sep=";", index_col=0)
multifield = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/multifield.0099-normcols.bin-200000.data',skiprows=4, sep=";", index_col=0)
phy = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/phy.data',skiprows=4, sep=";", index_col=0)
shuttle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/shuttle_trn_corr-normcols.data',skiprows=4, sep=";", index_col=0)

Data set projetado

In [ ]:
proj_color_histogram, proj_mammals, proj_fibers, proj_multifield, proj_phy, proj_shuttle = [], [], [], [], [], []

for i in range(10):
  proj_color_histogram.append(pd.DataFrame(LAMPx(color_histogram.to_numpy())))
  proj_mammals.append(pd.DataFrame(LAMPx(mammals.to_numpy())))
  proj_fibers.append(pd.DataFrame(LAMPx(fibers.to_numpy())))
  proj_multifield.append(pd.DataFrame(LAMPx(multifield.to_numpy())))
  proj_phy.append(pd.DataFrame(LAMPx(phy.to_numpy())))
  proj_shuttle.append(pd.DataFrame(LAMPx(shuttle.to_numpy())))

In [ ]:
def salvar_arquivo(nome, valor, data_set, tempo):
    nome = nome + '.txt'
    valor = str(round(valor, 4))
    tempo = str(round(tempo, 4))
    arquivo = open('/content/drive/MyDrive/Colab Notebooks/Resultados/'+ nome, 'a')
    arquivo.write(data_set + ',' + tempo + ',' + valor +'\n')
    arquivo.close()

##Funções para salvar os arquivos

Metricas de qualidade
# Trustworthiness



A confiabilidade de Mt resulta em  valores entre 0 e 1, onde 1 é o caso ideial. Ela mede a proporção de pontos do espaço N que não estão em conjunto dos vizinhos no 2D.

Onde N é igual ao número de dimensões do conjuto de dados, 
K é um valor arbirtrário dado pelo usuario que será utilizado para o calculo do numero de vizinhos para um ponto, 
n é o número de amostras(quantidade de dados a serem analisados), e Ui(k) é um conjunto de dados com os vizinhos de um ponto i no 2D mas que não estão entre os K vizinhos do ND(conjunto de dados), r(i, j) é a classificação do ponto no 2D, j é um conjunto ordenado dos vizinhos mais proximos do ponto i, e i é um ponto do conjunto.


$Mt = 1 - \frac{2}{NK(2n - 3K - 1)} \sum\limits_{i=1}^{N}\sum\limits_{j \epsilon Ui(K)}(r(i,j) - K)$

original 

Trustworthiness Mt: With values in [0, 1], with 1 being
the best, this measures the proportion of points in D that
are also close in P(D) [20]. A related metric measures
the false neighbors of a projected point [97]. Mt tells how
much one can trust that local patterns in a projection,
e.g., clusters, represent actual patterns in the data. In the
definition (Tab. 5), U
(K)
i
is the set of points that are among
the K nearest neighbors of point i in the 2D space but not
among the K nearest neighbors of point i in R
n; and r(i, j)
is the rank of the 2D point j in the ordered set of nearest
neighbors of i in 2D. We chose K = 7 for this study, in line
with [13], [98];

## codigo Rafael

In [ ]:
import numpy as np

def trustworthiness_o(D_high, D_low, k=7):

    n = D_high.shape[0]
    

    nn_orig = np.argsort(D_high)
    nn_proj = np.argsort(D_low)

    
    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        U = np.setdiff1d(knn_proj[i], knn_orig[i])

        sum_j = 0
        for j in range(U.shape[0]):
            sum_j += np.where(nn_orig[i] == U[j], nn_orig[i], U[j])[0] - k

        sum_i += sum_j

    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)))


## Codigo paralelizado

In [ ]:
##@njit
def trustworthiness_p(D_high, D_low, k=7):
    n = D_high.shape[0]
    

    nn_orig = np.argsort(D_high)
    nn_proj = np.argsort(D_low)

    
    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
      U = np.setdiff1d(knn_proj[i], knn_orig[i])
      sum_i += second_calc(nn_orig[i], U, k)

    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)))

@njit
def second_calc(nn_orig, U, k):
    sum_j = 0
    for j in range(U.shape[0]):
        sum_j += np.where(nn_orig == U[j], nn_orig, U[j])[0] - k

    return sum_j

# Continuity

A confiabilidade de Mc resulta em valores entre 0 e 1, onde 1 é o caso ideial. Ela mede a proporção de vizinhos de um ponto, onde relaciona a quantidade vizinhos ausentes de um ponto no espaço projetado.

Onde N é igual ao número de dimensões do conjuto de dados, K é um valor arbirtrário dado pelo usuario que será utilizado para o calculo do numero de vizinhos para um ponto, n é o número de amostras(quantidade de dados a serem analisados), e Vi(k) é um conjunto de dados com os vizinhos de um ponto i no Rn(espaço original) mas que não estão entre os K vizinhos do 2D, r(i, j) é a classificação do ponto no Rn, j é um conjunto ordenado dos vizinhos mais proximos do ponto i no Rn, e i é um ponto do conjunto.


$Cn = 1 - \frac{2}{NK(2n - 3K - 1)} \sum\limits_{i=1}^{N}\sum\limits_{j \epsilon Vi(K)}(r^-(i,j) - K)$

original

Continuity Mc: With values in [0, 1], with 1 being the
best, this measures the proportion of points in P(D) that
are also close together in D [20]. It is closely related to
the missing neighbors of a projected point [97]. In the
definition (Tab. 5), V
(K)
i
is the set of points that are among
the K nearest neighbors of point i in R
n but not among the
K nearest neighbors in 2D; and rˆ(i, j) is the rank of the R
n
point j in the ordered set of nearest neighbors of i in R
n.
As for Mt, we chose K = 7.


## codigo

###Paralelizado

In [ ]:
def continuity_p(D_high, D_low, k=7):
    
    n = D_high.shape[0]

    nn_orig = D_high.argsort()
    nn_proj = D_low.argsort()

    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        V = np.setdiff1d(knn_orig[i], knn_proj[i])
        sum_i += second_calc(nn_proj[i], V, k)

    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)))

@njit
def second_calc(nn_proj, V, k):
  sum_j = 0
  for j in range(V.shape[0]):
      sum_j += np.where(nn_proj == V[j], nn_proj, V[j])[0] - k

  return sum_j


###Rafael

In [ ]:
def continuity(D_high, D_low, k=7):
    
    n = D_high.shape[0]

    nn_orig = D_high.argsort()
    nn_proj = D_low.argsort()

    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        V = np.setdiff1d(knn_orig[i], knn_proj[i])

        sum_j = 0
        for j in range(V.shape[0]):
            sum_j += np.where(nn_proj[i] == V[j], nn_proj[i], V[j])[0] - k
        #print(sum_j)
        sum_i += sum_j
    
    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)))





#Stress de kruskal

##Sress paralelizado

In [ ]:
@vectorize([float64(float64, float64), float64(float64, float64), float64(float64, float64), float64(float64, float64)])
def top_p(Matriz_Dist, Matriz_proj):
    return (Matriz_Dist - Matriz_proj) ** 2


@njit(fastmath=True, nogil=True)
def somatorio_p(Matriz_o, Matriz_p):
    print('calculando stress paralelizado')
    '''Matriz_o: matriz do espaço original
    Matriz_p: matriz do espaço projetado'''
    somab = 0  # somatorio da parte inferior do calculo do stress de kruskal
    somat = 0  # somatorio da parte superior do calculo do stress de kruskal
    shape = np.shape(Matriz_o)  # vai pegar a quantidade de linha e colunas respectivamente
    for i in range(0, shape[0]):
        result = calculation_parts_p(Matriz_o, Matriz_p, i, i + 1, shape[0])
        somab += result[0]
        somat += result[1]

    return somat / somab


@njit(fastmath=True, nogil=True)
def calculation_parts_p(Mo, Mp, stance, init, end):
    '''
    Mo: matriz do espaço original
    Mp: matriz do espaço projetado
    stance: equivale o primeiro i do somatorio, onde ele é fixo até terminar o segundo somatorio
    init: equivale a o i do segundo somatorio, onde ele inicia i anterior mais 1 para nao repetir calculo
    end: o valor final de i que pode ser alcançado
    columns: são o número de colunas da matriz original, -2 pq o primeiro e o ultimo valor da linha não entra no calculo
    e o columns sem isso teria a quantidade de colunas
    '''
    somab = 0
    somat = 0
    for j in range(init, end):  # isto serve para não calcular linhas anteriores já calculdas

        # o primeiro parametro da linha não entrar no calculo pq é o indice da matriz e o ultimo era de classificação
        distanciaO = (np.sum(top_p(Mo[stance], Mo[j]))) ** 0.5
        # o mesmo caso anterior do commit anterior
        distanciap = (np.sum(top_p(Mp[stance], Mp[j]))) ** 0.5

        somab += distanciaO ** 2  # da formula onde a distancia do espaço projetado na parte inferior é elevado ao
        # quadrado
        somat += top_p(distanciaO, distanciap)

    return somab, somat


##Stress original

In [ ]:
def top(Matriz_Dist, Matriz_proj):
    return (Matriz_Dist - Matriz_proj) ** 2

def somatorio(Matriz_o, Matriz_p):
    #print('calculando stress não paralelizado')
    '''Matriz_o: matriz do espaço original
    Matriz_p: matriz do espaço projetado'''
    somab = 0  # somatorio da parte inferior do calculo do stress de kruskal
    somat = 0  # somatorio da parte superior do calculo do stress de kruskal
    shape = np.shape(Matriz_o)  # vai pegar a quantidade de linha e colunas respectivamente
    for i in range(0, shape[0]):
        result = calculation_parts(Matriz_o, Matriz_p, i, i + 1, shape[0], shape[1] - 2)
        somab += result[0]
        somat += result[1]

    return somat / somab


def calculation_parts(Mo, Mp, stance, init, end, columns):
    '''
    Mo: matriz do espaço original
    Mp: matriz do espaço projetado
    stance: equivale o primeiro i do somatorio, onde ele é fixo até terminar o segundo somatorio
    init: equivale a o i do segundo somatorio, onde ele inicia i anterior mais 1 para nao repetir calculo
    end: o valor final de i que pode ser alcançado
    columns: são o número de colunas da matriz original, -2 pq o primeiro e o ultimo valor da linha não entra no calculo
    e o columns sem isso teria a quantidade de colunas
    '''
    somab = 0
    somat = 0
    for j in range(init, end):  # isto serve para não calcular linhas anteriores já calculdas

        # o primeiro parametro da linha não entrar no calculo pq é o indice da matriz e o ultimo era de classificação
        distanciaO = (np.sum(top(Mo[stance][1:columns], Mo[j][1:columns]))) ** 0.5
        # o mesmo caso anterior do commit anterior
        distanciap = (np.sum(top(Mp[stance][1:3], Mp[j][1:3]))) ** 0.5

        somab += distanciaO ** 2  # da formula onde a distancia do espaço projetado na parte inferior é elevado ao
        # quadrado
        somat += top(distanciaO, distanciap)

    return somab, somat


#Ambiente de Testes

##testes TW

###TW original

####mammals

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_o(mammals.to_numpy(),proj_mammals[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW original', valor, 'mammals', tempo_final)

####Color Histogram

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_o(color_histogram.to_numpy(),proj_color_histogram[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW original', valor, 'Color_Histogram', tempo_final)

####Fibers

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_o(fibers.to_numpy(),proj_fibers[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW original', valor, 'Fibers', tempo_final)

####Multifield

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_o(multifield.to_numpy(),proj_multifield[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW original', valor, 'Multifield', tempo_final)

####Shuttle

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_o(shuttle.to_numpy(),proj_shuttle[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW original', valor, 'Shuttle', tempo_final)

####Phy

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_o(phy.to_numpy(),proj_phy[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW original', valor, 'Phy', tempo_final)

###TW paralelizado

####Mammals

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_p(mammals.to_numpy(),proj_mammals[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW paralelizado', valor, 'mammals', tempo_final)

####Color_Histogram

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_p(color_histogram.to_numpy(),proj_color_histogram[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW paralelizado', valor, 'Color_Histogram', tempo_final)

####Fibers

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_p(fibers.to_numpy(),proj_fibers[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW paralelizado', valor, 'Fibers', tempo_final)

####Multifield

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_p(multifield.to_numpy(),proj_multifield[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW paralelizado', valor, 'Multifield', tempo_final)

####Shuttle

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_p(shuttle.to_numpy(),proj_shuttle[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW paralelizado', valor, 'Shuttle', tempo_final)

####Phy

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = trustworthiness_p(phy.to_numpy(),proj_phy[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('TW paralelizado', valor, 'Phy', tempo_final)

##Teste CN

###CN original

####Mammals

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity(mammals.to_numpy(),proj_mammals[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity original', valor, 'mammals', tempo_final)

####Color_Histogram

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity(color_histogram.to_numpy(),proj_color_histogram[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity original', valor, 'Color_Histogram', tempo_final)

####Fibers

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity(fibers.to_numpy(),proj_fibers[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity original', valor, 'Fibers', tempo_final)

####Multifield

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity(multifield.to_numpy(),proj_multifield[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity original', valor, 'Multifield', tempo_final)

####Shuttle

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity(shuttle.to_numpy(),proj_shuttle[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity original', valor, 'Shuttle', tempo_final)

####Phy

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity(phy.to_numpy(),proj_phy[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity original', valor, 'Phy', tempo_final)

###CN paralelizado

####Mammals

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity_p(mammals.to_numpy(),proj_mammals[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity paralelizado', valor, 'mammals', tempo_final)

####Color_Histogram

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity_p(color_histogram.to_numpy(),proj_color_histogram[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity paralelizado', valor, 'Color_Histogram', tempo_final)

####Fibers

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity_p(fibers.to_numpy(),proj_fibers[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity paralelizado', valor, 'Fibers', tempo_final)

####Multifield

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity_p(multifield.to_numpy(),proj_multifield[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity paralelizado', valor, 'Multifield', tempo_final)

####Shuttle

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity_p(shuttle.to_numpy(),proj_shuttle[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity paralelizado', valor, 'Shuttle', tempo_final)

####Phy

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = continuity_p(phy.to_numpy(),proj_phy[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('continuity paralelizado', valor, 'Phy', tempo_final)

##Teste Stress de Kruskal

###Stress não paralelizado

####mammals

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio(mammals.to_numpy(),proj_mammals[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress não paralelo', valor, 'mammals', tempo_final)

KeyboardInterrupt: ignored

####Color Histogram

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio(color_histogram.to_numpy(),proj_color_histogram[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress não paralelo', valor, 'Color_Histogram', tempo_final)

####Fibers

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio(fibers.to_numpy(),proj_fibers[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress não paralelo', valor, 'Fibers', tempo_final)

####Multifield

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio(multifield.to_numpy(),proj_multifield[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress não paralelo', valor, 'Multifield', tempo_final)

####Shuttle

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio(shuttle.to_numpy(),proj_shuttle[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress não paralelo', valor, 'Shuttle', tempo_final)

####Phy

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio(phy.to_numpy(),proj_phy[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress não paralelo', valor, 'Phy', tempo_final)

###Stress paralelizado

####mammals

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio_p(mammals.to_numpy(),proj_mammals[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress paralelo', valor, 'mammals', tempo_final)

####Color Histogram

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio_p(color_histogram.to_numpy(),proj_color_histogram[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress paralelo', valor, 'Color_Histogram', tempo_final)

####Fibers

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio_p(fibers.to_numpy(),proj_fibers[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress paralelo', valor, 'Fibers', tempo_final)

####Multifield

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio_p(multifield.to_numpy(),proj_multifield[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress paralelo', valor, 'Multifield', tempo_final)

####Shuttle

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio_p(shuttle.to_numpy(),proj_shuttle[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress paralelo', valor, 'Shuttle', tempo_final)

####Phy

In [ ]:
for i in range(10):

  tempo_inicio = time.time()
  valor = somatorio_p(phy.to_numpy(),proj_phy[i].to_numpy())
  tempo_final = time.time() - tempo_inicio

  salvar_arquivo('Stress paralelo', valor, 'Phy', tempo_final)